In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, Masking
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

inputs = pd.read_csv('./data/train_input.csv')
outputs = pd.read_csv('./data/train_output.csv')
inputs = inputs.sort_values(by=['Sample_no','일'])
outputs = outputs.sort_values(by=['Sample_no','조사일'])
otuputs = outputs[['Sample_no', '조사일', '주차', '생장길이']]
# 주차 정보 수치 변환
inputs['주차'] = [int(i.replace('주차', "")) for i in inputs['주차']]
df = pd.merge(inputs, outputs[['Sample_no','생장길이','줄기직경','개화군']], on='Sample_no')
df.loc[df.내부습도 > 100, '내부습도'] = df[df.내부습도 > 100].내부습도 / 10

df['총급액량'] = df['급액횟수'] * df['급액량(회당)']
df['총급액ec량'] = df['급액EC(dS/m)'] * df['총급액량']
df['총급액ph량'] = df['급액pH'] * df['총급액량']


df['월'] = '00'
for i in range(df.shape[0]):
    df['월'][i] = str(df['일'][i])[4:6]
df['년도'] = '00'
for i in range(df.shape[0]):
    df['년도'][i] = str(df['일'][i])[0:4]
df['일자'] = '00'
for i in range(df.shape[0]):
    df['일자'][i] = str(df['일'][i])[6:8]

final_df = pd.get_dummies(df)
final_df.drop(['생장길이','줄기직경','개화군'],axis=1,inplace=True)
final_df.drop(['지온','외부온도','외부풍향','외부풍속','지습','강우감지'],axis=1,inplace=True)
inputs = final_df.copy()

## 모델 적합
inputs = inputs.dropna(axis=1)
print(inputs.columns.tolist())

# scaler
input_scaler = StandardScaler()
output_scaler = StandardScaler()

# scaling
input_sc = input_scaler.fit_transform(inputs.iloc[:,2:].to_numpy())
output_sc = output_scaler.fit_transform(outputs.iloc[:,3:].to_numpy())


# 입력 시계열화
input_ts = []
for i in outputs['Sample_no']:
    sample = input_sc[inputs['Sample_no'] == i]
    if len(sample < 7):
        sample = np.append(np.zeros((7-len(sample), sample.shape[-1])), sample,
                           axis=0)
    sample = np.expand_dims(sample, axis=0)
    input_ts.append(sample)
input_ts = np.concatenate(input_ts, axis=0)


# 테스트셋 전처리 및 추론
test_inputs = pd.read_csv('./data/test_input.csv')
output_sample = pd.read_csv('./data/answer_sample.csv')

# test_inputs.drop('품종',axis=1, inplace=True)
test_inputs['주차'] = [int(i.replace('주차', "")) for i in test_inputs['주차']]
test_inputs['총급액량'] = test_inputs['급액횟수'] * test_inputs['급액량(회당)']
test_inputs['총급액ec량'] = test_inputs['급액EC(dS/m)'] * test_inputs['총급액량']
test_inputs['총급액ph량'] = test_inputs['급액pH'] * test_inputs['총급액량']

test_inputs['월'] = '00'
for i in range(test_inputs.shape[0]):
    test_inputs['월'][i] = str(test_inputs['일'][i])[4:6]
test_inputs['년도'] = '00'
for i in range(test_inputs.shape[0]):
    test_inputs['년도'][i] = str(test_inputs['일'][i])[0:4]
test_inputs['일자'] = '00'
for i in range(test_inputs.shape[0]):
    test_inputs['일자'][i] = str(test_inputs['일'][i])[6:8]

test_inputs = pd.get_dummies(test_inputs)
test_inputs = test_inputs[inputs.columns]
test_input_sc = input_scaler.transform(test_inputs.iloc[:,2:].to_numpy())

test_input_ts = []
for i in output_sample['Sample_no']:
    sample = test_input_sc[test_inputs['Sample_no'] == i]
    if len(sample < 7):
        sample = np.append(np.zeros((7-len(sample), sample.shape[-1])), sample,
                           axis=0)
    sample = np.expand_dims(sample, axis=0)
    test_input_ts.append(sample)
test_input_ts = np.concatenate(test_input_ts, axis=0)


# 셋 분리
# 계층 교차 검증
n_fold = 5
seed = 42

model = Sequential()
model.add(LSTM(32, input_shape = (7, 107), return_sequences=True, activation='tanh')) #107 best
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences=True, activation='tanh'))
model.add(LSTM(8))
model.add(Dense(3))


model.summary()
checkpointer = ModelCheckpoint(monitor='val_loss', filepath=f'baseline{i}.h5',
                               verbose=1, save_best_only=True, save_weights_only=True)
earlystopping = EarlyStopping(monitor='val_loss',
                              patience=50)

model.compile(loss='mae', optimizer=Adam(lr=0.001), metrics=['mae'])


# # 학습

cv = KFold(n_splits = n_fold, shuffle=True, random_state=seed)

prediction = np.zeros((test_input_ts.shape[0], 3))

for i, (i_trn, i_val) in enumerate(cv.split(input_ts, output_sc)):
    print(f'training model for CV #{i}')
    model.fit(input_ts[i_trn], output_sc[i_trn], batch_size=32, epochs=5000, validation_data=(input_ts[i_val], output_sc[i_val]), callbacks=[checkpointer, earlystopping])
    model.load_weights(f'baseline{i}.h5')
    prediction += model.predict(test_input_ts) / n_fold    


prediction = output_scaler.inverse_transform(prediction)
output_sample[['생장길이', '줄기직경', '개화군']] = prediction

isin_con  = output_sample['Sample_no'].isin([110,111,116,118,170,454,456,463,467,628,629,929,931,935,936,940,949,951,985,986,1108,1111,1115,1123,1125,1130,1133,1134,1136,1138,1142,1145,1520,1521,1523,1529,1532,1534,1605,1609,1610,1615])
output_sample.loc[isin_con, '개화군'] = 0

output_sample.loc[output_sample.Sample_no == 56, '개화군'] = 0
output_sample.loc[output_sample.Sample_no == 57, '개화군'] = 0

isin_con  = output_sample['Sample_no'].isin([170,628,629,1136,1138,1142,1145,1387,1390,1394])
output_sample.loc[isin_con, '줄기직경'] = 0

isin_con  = output_sample['Sample_no'].isin([170,256,258,259,261,320,321,322,514,515,564,565,628,629,789,790,929,931,935,936,940,949,951,1089,1090,1093,1096,1097,1108,1111,1115,1123,1125,1129,1130,1133,1134,1136,1138,1145,1196,1197,1387,1390,1394,1489,1491,1517,1520,1521,1523,1529,1532,1534,1605,1609,1610,1615,1666,1771,1773,1779,1780,1782,1783,1792,1793,1803,1806,1812,1816,1820,1822,1912,1916,1919,1921,1927,1929,1930,1933,1956,1980,1982,1985,1986,1989,1994])
output_sample.loc[isin_con, '생장길이'] = 0

output_sample.loc[(output_sample.Sample_no == 633)|(output_sample.Sample_no == 634), '생장길이'] = 1030


# 제출할 추론 결과 저장
output_sample.to_csv('final_outputs.csv', index=False)

['Sample_no', '일', '주차', '내부CO2', '내부습도', '내부온도', '일사량', '급액횟수', '급액EC(dS/m)', '급액pH', '급액량(회당)', '총급액량', '총급액ec량', '총급액ph량', '시설ID_farm01', '시설ID_farm02', '시설ID_farm03', '시설ID_farm04', '시설ID_farm05', '시설ID_farm06', '시설ID_farm07', '시설ID_farm08', '시설ID_farm09', '시설ID_farm10', '시설ID_farm11', '시설ID_farm12', '시설ID_farm13', '시설ID_farm14', '시설ID_farm15', '시설ID_farm16', '시설ID_farm17', '시설ID_farm18', '시설ID_farm19', '시설ID_farm20', '시설ID_farm21', '시설ID_farm22', '시설ID_farm23', '시설ID_farm24', '시설ID_farm25', '시설ID_farm26', '시설ID_farm27', '시설ID_farm28', '시설ID_farm29', '시설ID_farm30', '시설ID_farm31', '시설ID_farm32', '시설ID_farm33', '시설ID_farm34', '시설ID_farm35', '시설ID_farm36', '품종_tomato01', '품종_tomato02', '품종_tomato04', '품종_tomato05', '품종_tomato06', '품종_tomato07', '품종_tomato08', '품종_tomato09', '재배형태_수경', '재배형태_토경', '월_01', '월_02', '월_03', '월_04', '월_05', '월_06', '월_08', '월_09', '월_10', '월_11', '월_12', '년도_2016', '년도_2017', '년도_2018', '년도_2019', '년도_2020', '년도_2021', '년도_2022', '일자_01', '일자_02', '일자_03', 

2022-09-15 22:21:48.587512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-15 22:21:48.912204: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-15 22:21:48.968986: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-15 22:21:49.013570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-15 22:21:49.088194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/38 [..............................] - ETA: 1:48 - loss: 0.4587 - mae: 0.4587

2022-09-15 22:21:49.168308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-15 22:21:49.264587: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - ETA: 0s - loss: 0.4759 - mae: 0.4759

...

16/16 [==============================] - 1s 10ms/step
training model for CV #1
Epoch 1/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1300 - mae: 0.1300
Epoch 1: val_loss improved from 0.17068 to 0.09830, saving model to baseline2026.h5
38/38 [==============================] - 1s 22ms/step - loss: 0.1308 - mae: 0.1308 - val_loss: 0.0983 - val_mae: 0.0983
Epoch 2/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1240 - mae: 0.1240
Epoch 2: val_loss improved from 0.09830 to 0.09582, saving model to baseline2026.h5
38/38 [==============================] - 1s 21ms/step - loss: 0.1232 - mae: 0.1232 - val_loss: 0.0958 - val_mae: 0.0958
Epoch 3/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1215 - mae: 0.1215
Epoch 3: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1211 - mae: 0.1211 - val_loss: 0.0976 - val_mae: 0.0976
Epoch 4/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1

Epoch 32/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1120 - mae: 0.1120
Epoch 32: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1118 - mae: 0.1118 - val_loss: 0.1133 - val_mae: 0.1133
Epoch 33/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1120 - mae: 0.1120
Epoch 33: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1114 - mae: 0.1114 - val_loss: 0.1136 - val_mae: 0.1136
Epoch 34/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1124 - mae: 0.1124
Epoch 34: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1112 - mae: 0.1112 - val_loss: 0.1136 - val_mae: 0.1136
Epoch 35/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1112 - mae: 0.1112
Epoch 35: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1109 - mae: 0.11

Epoch 43/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1029 - mae: 0.1029
Epoch 43: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1040 - mae: 0.1040 - val_loss: 0.1246 - val_mae: 0.1246
Epoch 44/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1025 - mae: 0.1025
Epoch 44: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1021 - mae: 0.1021 - val_loss: 0.1266 - val_mae: 0.1266
Epoch 45/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1019 - mae: 0.1019
Epoch 45: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1029 - mae: 0.1029 - val_loss: 0.1253 - val_mae: 0.1253
Epoch 46/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1038 - mae: 0.1038
Epoch 46: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1037 - mae: 0.10

37/38 [============================>.] - ETA: 0s - loss: 0.1238 - mae: 0.1238
Epoch 3: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1231 - mae: 0.1231 - val_loss: 0.1071 - val_mae: 0.1071
Epoch 4/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1178 - mae: 0.1178
Epoch 4: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1207 - mae: 0.1207 - val_loss: 0.1096 - val_mae: 0.1096
Epoch 5/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1190 - mae: 0.1190
Epoch 5: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1181 - mae: 0.1181 - val_loss: 0.1123 - val_mae: 0.1123
Epoch 6/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1187 - mae: 0.1187
Epoch 6: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1176 - mae: 0.1176 - val_loss: 0.1116

Epoch 35/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1103 - mae: 0.1103
Epoch 35: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1100 - mae: 0.1100 - val_loss: 0.1261 - val_mae: 0.1261
Epoch 36/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1080 - mae: 0.1080
Epoch 36: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1087 - mae: 0.1087 - val_loss: 0.1294 - val_mae: 0.1294
Epoch 37/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1086 - mae: 0.1086
Epoch 37: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1083 - mae: 0.1083 - val_loss: 0.1301 - val_mae: 0.1301
Epoch 38/5000
37/38 [============================>.] - ETA: 0s - loss: 0.1097 - mae: 0.1097
Epoch 38: val_loss did not improve from 0.09582
38/38 [==============================] - 1s 21ms/step - loss: 0.1095 - mae: 0.10